In [210]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import os
%matplotlib notebook 
# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
    'text.usetex': True,  # Enable LaTeX for text rendering
    'text.latex.preamble': r'\usepackage{amsmath}',
    'font.family': 'serif',  # Use a serif font family
    'font.serif': 'Palatino',  # Set Palatino as the serif font
    'font.size': 20,  # Font size for general text
    'axes.titlesize': 20,  # Font size for axis titles
    'axes.labelsize': 20,  # Font size for axis labels
    'xtick.labelsize': 20,  # Font size for x-axis tick labels
    'ytick.labelsize': 20,  # Font size for y-axis tick labels
    'legend.fontsize': 20,  # Font size for legends
    'figure.figsize': [8, 6],  # Size of the plot (width x height)
    'figure.autolayout': True,  # Automatic layout adjustment
    'savefig.format': 'svg',  # Default format for saving figures
    'figure.facecolor': 'none',  # Make the figure face color transparent
    'axes.facecolor': 'none',  # Make the axes face color transparent
    'savefig.transparent': True  # Save figures with transparent background
})

output_dir = r"C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Latex\Figures_From_Python"
os.makedirs(output_dir, exist_ok=True)
name = 'Master_CYL_TOPO'
########################################                 Define constants                   #############################################
fixed_lam = 1
fixed_gamma = 1
k_a = 2 * np.pi / fixed_lam
distance = 0.2 * fixed_lam

# Set the system parameters
n_rings = 5
n_chains = 13  # number of chains
N_atoms = n_chains * n_rings  # number of atoms

theta = 0#np.pi / 2


########################################               define the geometry                 #############################################
def dipole_vector(phi, theta):
    """Returns the dipole vector given its orientation by angles phi and theta."""  #
    dipole = np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])
    return dipole


def chain_positions(distance, N_atoms):
    Pos = np.zeros((N_atoms, 3))
    for i in range(N_atoms):
        Pos[i, 2] = i * distance
    return Pos


def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]])


def ring_positions(distance, n_chains):
    Pos = np.zeros((n_chains, 3))
    dphi = 2 * np.pi / n_chains
    if n_chains == 1:
        radius = 0
    else:
        radius = distance / 2 / np.sin(np.pi / n_chains)
    helper = np.array([radius, 0, 0])
    for i in range(n_chains):
        rotation_matrix = z_rotation(dphi * i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos


def cyl_positions(distance, N_atoms, n_chains):
    Pos = np.zeros((N_atoms, 3))
    Pos_chain = chain_positions(distance, N_atoms // n_chains)
    Pos_ring = ring_positions(distance, n_chains)
    for i in range(n_chains):
        Pos[i * (N_atoms // n_chains): (i + 1) * (N_atoms // n_chains)] = Pos_chain + Pos_ring[i]
    return Pos


def cyl_dipoles(N_atoms, n_chains, theta):
    dipoles_chain = [dipole_vector(0, theta)] * (N_atoms // n_chains)
    dipoles = np.zeros((N_atoms, 3))
    dphi = 2 * np.pi / n_chains
    for i in range(n_chains):
        rotation_matrix = z_rotation(dphi * i)
        for j in range(N_atoms // n_chains):
            dipoles[i * (N_atoms // n_chains) + j] = np.matmul(rotation_matrix, dipoles_chain[j])
    return dipoles


pos = cyl_positions(distance, N_atoms, n_chains)
#Dipoles = [dipole_vector(1, 0) for _ in range(N_atoms)] # = (0,0,1): (x,y,z)
dipoles = cyl_dipoles(N_atoms, n_chains, theta)

# Plotting 
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
# Plot the positions 
ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], c='b', marker='o')

# Plot the dipoles as arrows 
for i in range(N_atoms):
    ax.quiver(pos[i, 0], pos[i, 1], pos[i, 2], dipoles[i, 0], dipoles[i, 1], dipoles[i, 2], color='r', length=0.5)
    # Set labels and title 
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('positions and dipoles')
plt.show()

<IPython.core.display.Javascript object>

In [211]:
########################################             create the Hamiltonean                 #################################################
def Green_tensor(r_a, r_b):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
            * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
               + (- kappa ** 2 - 3 * 1j * kappa + 3)
               * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance, dipoles, N_atoms, n_chains):
    positions = cyl_positions(distance, N_atoms, n_chains)
    G_matrix = np.zeros((N_atoms, N_atoms), dtype=complex)
    for a in range(N_atoms):
        for b in range(N_atoms):
            G_matrix[a, b] = fixed_gamma
            r_a, r_b = positions[a], positions[b]
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                G_matrix[a, b] = np.imag((6 * np.pi * fixed_gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b), d_b.T))))
    return G_matrix

def V_matrix(distance, dipoles, N_atoms, n_chains):
    positions = cyl_positions(distance, N_atoms, n_chains)
    V_matrix = np.zeros((N_atoms, N_atoms), dtype=complex)
    for a in range(N_atoms):
        for b in range(N_atoms):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                V_matrix[a, b] = - np.real((3 * np.pi * fixed_gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b), d_b.T))))
    return V_matrix
    
def H_eff(distance, dipoles, N_atoms, n_chains):
    G = Gamma_matrix(distance, dipoles, N_atoms, n_chains)
    V = V_matrix(distance, dipoles, N_atoms, n_chains)
    return - k_a * qeye(N_atoms) +  Qobj(V) - 1j / 2 * Qobj(G)

H = H_eff(distance, dipoles, N_atoms, n_chains)

H

Quantum object: dims=[[65], [65]], shape=(65, 65), type='oper', dtype=Dense, isherm=False
Qobj data =
[[-6.28318531-0.5j        -1.13697987-0.42536824j -0.06314074-0.24765652j
  ...  0.051918  -0.15852926j  0.08608729-0.02537609j
   0.03715699+0.03938818j]
 [-1.13697987-0.42536824j -6.28318531-0.5j        -1.13697987-0.42536824j
  ... -0.05929179-0.29625804j  0.051918  -0.15852926j
   0.08608729-0.02537609j]
 [-0.06314074-0.24765652j -1.13697987-0.42536824j -6.28318531-0.5j
  ...  0.384059  -0.35493593j -0.05929179-0.29625804j
   0.051918  -0.15852926j]
 ...
 [ 0.051918  -0.15852926j -0.05929179-0.29625804j  0.384059  -0.35493593j
  ... -6.28318531-0.5j        -1.13697987-0.42536824j
  -0.06314074-0.24765652j]
 [ 0.08608729-0.02537609j  0.051918  -0.15852926j -0.05929179-0.29625804j
  ... -1.13697987-0.42536824j -6.28318531-0.5j
  -1.13697987-0.42536824j]
 [ 0.03715699+0.03938818j  0.08608729-0.02537609j  0.051918  -0.15852926j
  ... -0.06314074-0.24765652j -1.13697987-0.42536824j
  -6

In [212]:
t_max = 100
times = np.linspace(0, t_max, 100) # list of times
# evolution
psiini = basis(N_atoms, 3)

In [213]:
def survival_probabilities(times, distance, dipoles, Psi_0, N_atoms, n_chains):
    H = H_eff(distance, dipoles, N_atoms, n_chains)
    coeffs_sq_mods  = np.zeros((len(times), N_atoms))
    P_surs  = np.zeros(len(times))
    for t_idx, t in enumerate(times):
        U = (-1j * H * t).expm()
        Psi_t = (U * Psi_0).full().flatten()
        Probs = np.abs(Psi_t)**2
        coeffs_sq_mods[t_idx, :] = Probs
        P_surs[t_idx] = Probs.sum()
    return coeffs_sq_mods, P_surs

def k_wave_packet(N_atoms):
    coefficients = np.exp(- (k_values - k_s) ** 2 / (4 * sigma_k ** 2))
    coefficients = coefficients / np.linalg.norm(coefficients)
    wave_packet_k = np.zeros(N_atoms, dtype=complex)
    
    for k, a_k in enumerate(coefficients):
        wave_packet_k += a_k * FT[:, k]

    wave_packet_k = wave_packet_k / np.linalg.norm(wave_packet_k)

    return wave_packet_k, coefficients

def real_space_wave_packet(N_atoms):
    coefficients = np.exp(- (k_values - k_s) ** 2 / (4 * sigma_k ** 2))

    # Do a phase shift to move the wavepacket in real space
    phase_shift = np.exp(1j * np.linspace(0, 2 * np.pi, N_atoms) * c * N_atoms)
    coefficients = coefficients * phase_shift

    wave_packet_real = np.zeros(N_atoms, dtype=complex)
    for alpha in range(N_atoms):
        a_alpha = 0
        for k, a_k in enumerate(coefficients):
            a_alpha += a_k * np.dot(np.eye(1, N_atoms, alpha, dtype=int).flatten(), FT[:,k])
        wave_packet_real[alpha] = a_alpha
    wave_packet_real = wave_packet_real / np.linalg.norm(wave_packet_real)
            
    return wave_packet_real

#
# PLOT THE Matrices in REAL and k space
#
def PLOT_REAL_and_K_MATRICES():
    plt.figure()
    im = plt.imshow(np.real(V), cmap='coolwarm', aspect='auto')
    max_value = np.max(np.real(V))
    min_value = np.min(np.real(V))
    
    cbar = plt.colorbar(im, label=r'$ V_{\alpha \beta} $')
    cbar.set_ticks([min_value, 0, max_value])
    cbar.set_ticklabels([f'{np.real(min_value):.1f}', '0', f'{np.real(max_value):.0f}'])
    
    plt.xlabel(r'$ \alpha $')
    plt.ylabel(r'$ \beta $')
    
    plt.xticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.yticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.gca().set_xticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    plt.gca().set_yticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    
    plt.show()
    
    plt.figure()
    im = plt.imshow(np.real(almost_D_V), cmap='coolwarm', aspect='auto')
    max_value = np.max(np.real(almost_D_V))
    min_value = np.min(np.real(almost_D_V))
    
    cbar = plt.colorbar(im, label=r'$ V_{k k^\prime} $')
    cbar.set_ticks([min_value, 0, max_value])
    cbar.set_ticklabels([f'{np.real(min_value):.1f}', '0', f'{np.real(max_value):.0f}'])
    
    plt.xticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.yticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.gca().set_xticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    plt.gca().set_yticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    
    plt.ylabel(r'$k$')
    plt.xlabel(r'$k^\prime$')
    

    plt.show()
    
    # Plot the G matrix (real space)
    plt.figure()
    im = plt.imshow(np.real(G), cmap='coolwarm', aspect='auto')
    max_value = np.max(np.real(G))
    min_value = np.min(np.real(G))
    
    cbar = plt.colorbar(im, label=r'$ G_{\alpha \beta} $')
    cbar.set_ticks([min_value, 0, max_value])
    cbar.set_ticklabels([f'{np.real(min_value):.1f}', '0', f'{np.real(max_value):.0f}'])
    
    plt.xlabel(r'$ \alpha $')
    plt.ylabel(r'$ \beta $')
    
    plt.xticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.yticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.gca().set_xticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    plt.gca().set_yticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    

    plt.show()
    
    # Compute almost_D_G matrix (momentum space for G matrix)
    plt.figure()
    im = plt.imshow(np.real(almost_D_G), cmap='coolwarm', aspect='auto')
    max_value = np.max(np.real(almost_D_G))
    min_value = np.min(np.real(almost_D_G))
    
    cbar = plt.colorbar(im, label=r'$ G_{k k^\prime} $')
    cbar.set_ticks([min_value, 0, max_value])
    cbar.set_ticklabels([f'{np.real(min_value):.1f}', '0', f'{np.real(max_value):.0f}'])
    
    plt.xticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.yticks([0, N_atoms // 3, 2 * N_atoms // 3, N_atoms - 1])
    plt.gca().set_xticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    plt.gca().set_yticklabels([r'1', f'{N_atoms//3}', f'{2*N_atoms//3}', f'{N_atoms}'])
    
    plt.ylabel(r'$k$')
    plt.xlabel(r'$k^\prime$')
    
    plt.show()
    
#
# PLOT THE SPECTRUK in k space, WAVEPACKET IN k & REAL spcae 
#
def Plot_SPECTRUM_and_PSI_0():
    _, psik = k_wave_packet(N_atoms)
    norm_squared = np.abs(psik.flatten()) ** 2

    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax1.scatter(k_values, eigenvalues_V, label=r'$ V_k $', color='#1f77b4', s=30, marker='o', alpha=0.8, edgecolor='#0177C9', linewidth=1.5) # Spectrum
    ax1.scatter(k_values, eigenvalues_G, label=r'$ \Gamma_k $', color='#ff7f0e', s=30, marker='o', alpha=0.8, edgecolor='#FF7F02', linewidth=1.5)  # Decay modes
    ax1.axvline(k_s, linestyle='--', alpha=0.25, color='black')                           # center of the wavepacket in P space

    # Determine the range of p values within p(k_s) ± 2*sigma_k
    from scipy.stats import linregress  # Import linear regression function
    k_range_indices = (k_values >= (k_s - 2 * sigma_k)) & (k_values <= (k_s + 2 * sigma_k))
    k_values_range = k_values[k_range_indices]
    V_values_range = eigenvalues_V[k_range_indices]
    # for plotting
    k_range_indices_plot = (k_values >= (k_s - 10 * sigma_k)) & (k_values <= (k_s + 10 * sigma_k))
    k_values_range_plot = k_values[k_range_indices_plot]
    # Fit a line through these V values
    slope, intercept, _, _, _ = linregress(k_values_range, V_values_range)
    print("The slope v_g is", slope)
    fitted_line_short = slope * k_values_range + intercept
    fitted_line = slope * k_values_range_plot + intercept
    ax1.plot(k_values_range_plot, fitted_line, color='red', linestyle='-', linewidth=2)
    ax1.plot(k_values_range[0], fitted_line_short[0], marker='|', color='red', markersize=10, markeredgewidth=2)
    ax1.plot(k_values_range[-1], fitted_line_short[-1], marker='|', color='red', markersize=10, markeredgewidth=2)

    ax2 = ax1.twinx()
    ax2.plot(k_values, norm_squared, color='#2ca02c', linewidth=2.5, label=r'$ |\psi_k(0)|^2 $')     # P Space Wave packet
    ax2.annotate(r'$k_\text{s}$', xy=(k_s, 1.2 * np.max(norm_squared)), ha='left', va='center')

    # legends
    ax1.set_xlabel(r'$k$')
    ax1.set_ylabel(r'$\text{E}/ \gamma$') # Eigenvalues
    ax2.set_ylabel(r'$|c_k|^2$')
    ax1.legend(loc='best')
    ax2.legend(loc='lower left')

    # lims, ticks
    y1_min = min(np.min(eigenvalues_V), np.min(eigenvalues_G))
    y1_max = max(np.max(eigenvalues_V), np.max(eigenvalues_G))
#    ax1.set_ylim([round(y1_min, 0), round(y1_max, 0)])
#    ax1.set_yticks([round(y1_min, 0), 0, round(y1_max, 0)])

#    ax1.set_xlim(k_values[0], k_values[-1])
    xticks = [round(k_values[0], 1), 0, round(k_values[-1], 1)]
    sorted_by_closeness_to_one = sorted(range(len(eigenvalues_G)), key=lambda i: abs(eigenvalues_G[i] - 1))
    closest_two_indices = sorted_by_closeness_to_one[:2]  # Indices of the two eigenvalues closest to 1
    closest_two_k_values = [k_values[closest_two_indices[0]], k_values[closest_two_indices[1]]]
    xticks += [round(closest_two_k_values[0], 2), round(closest_two_k_values[1], 2)]  # Adding the two ticks, where G ~ 1
    xticks = sorted(xticks)
#    ax1.set_xticklabels([r'$-\pi / d_{\text{ext}}$', 'b', 0, 'b', r'$\pi / d_{\text{ext}}$'])
#    ax1.set_xticks(xticks)

#    ax2.set_ylim(0, round(2 * np.max(norm_squared), 1))
#    ax2.set_yticks([round(0, 0), round(2 * np.max(norm_squared), 1)])
#    ax2.set_yticklabels([0, round(2 * np.max(norm_squared), 1)])
    
    # safe
    plt.tight_layout()
    plt.show()


    # NOW IN REAL SPACE
    psix = real_space_wave_packet(N_atoms)
    print(np.linalg.norm(psix))
    norm_squared = np.abs(psix.flatten()) ** 2
    alphas = np.arange(N_atoms)

    plt.figure(figsize=(4,2))
    plt.plot(alphas, norm_squared, color='#2ca02c', linewidth=2.5, label=r'$ |\psi_x(0)|^2 $')

    # lims, ticks
    plt.xlabel(r'$ \alpha $')
    plt.ylabel('Amp')
    plt.ylim(0, 2 * np.max(norm_squared)) # → When plotting the |_|^2
    plt.xlim(0, N_atoms) # → When plotting the |_|^2
    plt.yticks([round(0, 0), round(2 * np.max(norm_squared), 1)])
    plt.xticks([0, N_atoms//3, 2*N_atoms//3, N_atoms])

    # safe
    plt.tight_layout()
#    plt.title('REAL - SPACE')
    ax1.legend(loc='best')

    plt.show()


def PLOT_REAL_EVO():
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, constrained_layout=True, figsize=(4, 4))

    im = ax1.imshow(y1.T, aspect='auto', origin='lower', cmap='coolwarm', extent=[0, t_max, 0, N_atoms])
    ax1.set_yticks([0, N_atoms//3, 2*N_atoms//3, N_atoms])
    ax1.set_yticklabels([1, N_atoms//3+1, 2*N_atoms//3+1, N_atoms])
    ax2.plot(times, y2, color='green', linewidth=2.5, label='collective')
    ax2.plot(times, np.exp(-times), 'r--', linewidth=2.5, label='1 atom')

    max_value = np.max(y1)
    cbar = fig.colorbar(im, ax=ax1, label=r'$ \vert c_{\alpha}(t)\vert ^2$')
    cbar.set_ticks([0, max_value])
    cbar.set_ticklabels([r'0', f'{max_value:.1f}'])

    ax2.set_ylim(0, 1.01)
    ax2.set_xticks([0, round(t_max/4), round(t_max/2), round(3* t_max/4), t_max])
    ax2.set_xticklabels(['0', f'{round(t_max/4)}', f'{round(t_max/2)}', f'{round(3*t_max/4)}', f'{round(t_max)}'])
    ax2.set_yticks([0, y2[-1], 1])
    ax2.set_yticklabels(['0', f'{y2[-1]:.2f}', '1'])
    ax1.set_ylabel(r'$\alpha$')
    ax2.set_xlabel(r'$\gamma t$')
    ax2.set_ylabel(r'$P_{\text{sur}}$')

    plt.show()

In [214]:
# THOSE determine the Shape of V_k, G_k 
k_values = np.linspace(- np.pi/distance, np.pi/distance, N_atoms)
# Construct the Fourier Transform matrix

In [221]:
#PLOT_REAL_and_K_MATRICES()

# PLOT
k_s = k_values[-round(N_atoms/2)]
#print(k_s / (np.pi / distance))
sigma_k = 0.02 * np.pi / distance
c = 1/2

FT = np.zeros((N_atoms, N_atoms), dtype=complex)
for alpha in range(N_atoms):
    FT[alpha, :] = np.exp(-1j * k_values * np.linalg.norm(pos[alpha]))
    FT[alpha, :] = FT[alpha, :]/ np.linalg.norm(FT[alpha, :])
    

# THOSE determine the Shape of V_k, G_k 
V = V_matrix(distance, dipoles, N_atoms, n_chains)
G = Gamma_matrix(distance, dipoles, N_atoms, n_chains)

almost_D_V = np.conjugate(FT.T) @ V @ FT
diagonal_entries_V = np.real(np.diag(almost_D_V))
almost_D_G = np.conjugate(FT.T) @ G @ FT
diagonal_entries_G = np.real(np.diag(almost_D_G))

eigenvalues_V = diagonal_entries_V
eigenvalues_G = diagonal_entries_G

Plot_SPECTRUM_and_PSI_0()

<IPython.core.display.Javascript object>

The slope v_g is 0.029697423807684135
1.0


<IPython.core.display.Javascript object>

In [222]:
#result = mesolve(H, psiini, times, [], [basis(N_atoms, i) * basis(N_atoms, i).dag() for i in range(N_atoms)], options={"store_states": True})

# y1: 2D array with shape (len(times), N_atoms) 
#y1 = np.array(result.expect).T 
# y2: 1D array with shape (len(times),) 
#y2 = np.sum(y1, axis=1)
psi0 = Qobj(real_space_wave_packet(N_atoms))


y1, y2 = survival_probabilities(times, distance, dipoles, psi0, N_atoms, n_chains)

In [223]:
PLOT_REAL_EVO()

<IPython.core.display.Javascript object>

In [99]:
#
# Analyze Reflection and Transmission coefficients
#

# Define time and indices
N_chain = N_atoms // 3
# Find the index closest to the target time
T = 10
time_idx = np.argmin(np.abs(times - T))
# Extract the probabilities at the initial time (time = 0)
initial_probabilities = y1[0, :]

# Extract the probabilities at the target time
probabilities_at_target_time = y1[time_idx, :]

# Calculate the initial probability in the first N_chain states (should be ~1 initially)
initial_prob_first_third = initial_probabilities[:N_chain].sum()

# Reflection coefficient: Probability in the first N_chain states at target time
prob_first_third = probabilities_at_target_time[:N_chain].sum()

# Transmission coefficient: Probability in the second N_chain states at target time
prob_second_third = probabilities_at_target_time[N_chain:2 * N_chain].sum()

prob_third_third = probabilities_at_target_time[2 * N_chain:].sum()

# Calculate the reflection and transmission coefficients as ratios
reflection_coeff = prob_first_third / initial_prob_first_third
transmission_coeff = prob_second_third / initial_prob_first_third

print(f"Reflection Coefficient at time {T}: {(reflection_coeff * 100):3f}")
print(f"Transmission Coefficient at time {T}: {(transmission_coeff * 100):3f}")
print(f"leackage into lower arm at {T}: {(prob_third_third / initial_prob_first_third * 100):3f}")

Reflection Coefficient at time 10: 21.956416
Transmission Coefficient at time 10: 72.215814
leackage into lower arm at 10: 1.278827
